In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

### Request for Census Tract Level Population data from Census API

In [3]:
stateCode = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16',
             '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
            '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', 
             '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']

In [4]:
len(stateCode)

51

In [5]:
import requests
import pandas as pd
from datetime import datetime
import time
import json
from io import StringIO

In [6]:
def collectAll_county(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs5?get=NAME,'+ variables + '&for=county:*&in=state:*&key=250f58e35283d400e4185b197e3abe9021079afe'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

#### Total population: B01003_001E

In [7]:
year = '2020'
variables = 'B01003_001E'

In [8]:
pop_19 = collectAll_county(year,variables)
pop_19.head()

['NAME', 'B01003_001E', 'state', 'county']


,county_name,B01003_001E,state code,county code
0,"Autauga County, Alabama",55639,01,001
1,"Baldwin County, Alabama",218289,01,003
2,"Barbour County, Alabama",25026,01,005
3,"Bibb County, Alabama",22374,01,007
4,"Blount County, Alabama",57755,01,009


In [9]:
pop_19.shape

(3221, 4)

#### 1.per capita income: B19301_001E

In [10]:
variables = 'B19301_001E'
df1 = collectAll_county(year,variables)

['NAME', 'B19301_001E', 'state', 'county']


In [11]:
df1.shape

(3221, 4)

In [12]:
df1 = df1.rename(columns = {'B19301_001E': 'per_capita_income'})
df1['fips'] = df1['state code'] + df1['county code']
df1.head()

,county_name,per_capita_income,state code,county code,fips
0,"Autauga County, Alabama",29804,01,001,01001
1,"Baldwin County, Alabama",33751,01,003,01003
2,"Barbour County, Alabama",20074,01,005,01005
3,"Bibb County, Alabama",22626,01,007,01007
4,"Blount County, Alabama",25457,01,009,01009


#### 2.Percent of Population Below Poverty Level: (C17002_002E+C17002_003E)/C17002_001E

In [13]:
variables = 'C17002_002E,C17002_003E,C17002_001E'
df2 = collectAll_county(year,variables)

['NAME', 'C17002_002E', 'C17002_003E', 'C17002_001E', 'state', 'county']


In [14]:
df2 = df2.dropna()
df2['C17002_002E'] = df2['C17002_002E'].astype('int')
df2['C17002_003E'] = df2['C17002_003E'].astype('int')
df2['C17002_001E'] = df2['C17002_001E'].astype('int')

In [15]:
df2['percent_below_poverty'] = ((df2['C17002_002E'] + df2['C17002_003E'])/df2['C17002_001E'])*100

In [16]:
df2['fips'] = df2['state code'] + df2['county code']
df2.head()

,county_name,C17002_002E,C17002_003E,C17002_001E,state code,county code,percent_below_poverty,fips
0,"Autauga County, Alabama",3504,4890,55181,01,001,15.211758,01001
1,"Baldwin County, Alabama",7250,12489,215170,01,003,9.173677,01003
2,"Barbour County, Alabama",3018,3294,22070,01,005,28.599909,01005
3,"Bibb County, Alabama",1875,1880,20748,01,007,18.098130,01007
4,"Blount County, Alabama",3834,4013,57127,01,009,13.736062,01009


In [17]:
df2.shape

(3221, 8)

#### 3.Percent of Population 25+ with Less than a 12th Grade Education: (B15002_003E,B15002_004E,B15002_005E,B15002_003E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E)/B15003_001E

In [18]:
def collectAll_county2(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs1?get=NAME,'+ variables + '&for=county:*&in=state:*'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

In [19]:
variables = 'B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E'
df3 = collectAll_county(year,variables)

['NAME', 'B15002_003E', 'B15002_004E', 'B15002_005E', 'B15002_006E', 'B15002_007E', 'B15002_008E', 'B15002_009E', 'B15002_010E', 'B15002_020E', 'B15002_021E', 'B15002_022E', 'B15002_023E', 'B15002_024E', 'B15002_025E', 'B15002_026E', 'B15002_027E', 'B15002_001E', 'state', 'county']


In [20]:
df3.shape

(3221, 20)

In [21]:
def changeInteger(value):
    if value != None:
        return int(value)
    else:
        return np.nan

In [22]:
df3['B15002_003E'] = df3['B15002_003E'].apply(changeInteger)
df3['B15002_004E'] = df3['B15002_004E'].apply(changeInteger)
df3['B15002_005E'] = df3['B15002_005E'].apply(changeInteger)
df3['B15002_006E'] = df3['B15002_006E'].apply(changeInteger)
df3['B15002_007E'] = df3['B15002_007E'].apply(changeInteger)
df3['B15002_008E'] = df3['B15002_008E'].apply(changeInteger)
df3['B15002_009E'] = df3['B15002_009E'].apply(changeInteger)
df3['B15002_010E'] = df3['B15002_010E'].apply(changeInteger)
df3['B15002_020E'] = df3['B15002_020E'].apply(changeInteger)
df3['B15002_021E'] = df3['B15002_021E'].apply(changeInteger)
df3['B15002_022E'] = df3['B15002_022E'].apply(changeInteger)
df3['B15002_023E'] = df3['B15002_023E'].apply(changeInteger)
df3['B15002_024E'] = df3['B15002_024E'].apply(changeInteger)
df3['B15002_025E'] = df3['B15002_025E'].apply(changeInteger)
df3['B15002_026E'] = df3['B15002_026E'].apply(changeInteger)
df3['B15002_027E'] = df3['B15002_027E'].apply(changeInteger)
df3['B15002_001E'] = df3['B15002_001E'].apply(changeInteger)

In [23]:
df3['percent_no_highschool_diploma'] = ((df3['B15002_003E']+df3['B15002_004E']+df3['B15002_005E']+df3['B15002_006E']+df3['B15002_007E']+df3['B15002_008E']+df3['B15002_009E']
                                       +df3['B15002_010E']+df3['B15002_020E']+df3['B15002_021E']+df3['B15002_022E']+df3['B15002_023E']+df3['B15002_024E']+df3['B15002_025E']+df3['B15002_003E']
                                       +df3['B15002_026E']+df3['B15002_027E'])/df3['B15002_001E'])*100

In [24]:
df3['fips'] = df3['state code'] + df3['county code']
df3.head()

,county_name,B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,...,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E,state code,county code,percent_no_highschool_diploma,fips
0,"Autauga County, Alabama",134,77,57,270,338,518,377,227,141,...,160,380,732,351,441,37860,01,001,11.640254,01001
1,"Baldwin County, Alabama",794,330,361,706,1012,2019,1329,1966,657,...,507,964,1108,1154,1437,155563,01,003,10.039020,01003
2,"Barbour County, Alabama",157,112,72,464,399,590,628,322,227,...,233,222,381,401,167,17797,01,005,26.150475,01005
3,"Bibb County, Alabama",197,13,129,213,251,399,614,173,126,...,234,36,268,142,195,15987,01,007,20.347783,01007
4,"Blount County, Alabama",507,296,499,504,324,623,535,473,298,...,335,399,715,513,306,39814,01,009,18.448285,01009


#### 4.Percent of Population Living in Mobile Homes: (B25033_006E+B25033_007E+B25033_012E+B25033_013E)/B25033_001E

In [25]:
variables = 'B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E'
df4 = collectAll_county(year,variables)

['NAME', 'B25033_006E', 'B25033_007E', 'B25033_012E', 'B25033_013E', 'B25033_001E', 'state', 'county']


In [26]:
df4.shape

(3221, 8)

In [27]:
df4['B25033_006E'] = df4['B25033_006E'].apply(changeInteger)
df4['B25033_007E'] = df4['B25033_007E'].apply(changeInteger)
df4['B25033_012E'] = df4['B25033_012E'].apply(changeInteger)
df4['B25033_013E'] = df4['B25033_013E'].apply(changeInteger)
df4['B25033_001E'] = df4['B25033_001E'].apply(changeInteger)

In [28]:
df4['percent_mobile_homes'] = ((df4['B25033_006E'] + df4['B25033_007E'] + df4['B25033_012E'] + df4['B25033_013E'])/df4['B25033_001E'])*100

In [29]:
df4['fips'] = df4['state code'] + df4['county code']
df4.head()

,county_name,B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E,state code,county code,percent_mobile_homes,fips
0,"Autauga County, Alabama",6837,97,2244,0,55061,01,001,16.668786,01001
1,"Baldwin County, Alabama",16598,289,6421,147,215335,01,003,10.892331,01003
2,"Barbour County, Alabama",4292,8,1818,0,22116,01,005,27.663230,01005
3,"Bibb County, Alabama",4802,18,1780,18,20717,01,007,31.944780,01007
4,"Blount County, Alabama",9454,76,3318,0,57191,01,009,22.465073,01009


#### 5.Percent of Population with No Vehicle Available: (B25044_003E+B25044_010E)/B25044_001E

In [30]:
variables = 'B25044_003E,B25044_010E,B25044_001E'
df5 = collectAll_county(year,variables)

['NAME', 'B25044_003E', 'B25044_010E', 'B25044_001E', 'state', 'county']


In [31]:
df5.shape

(3221, 6)

In [32]:
df5['B25044_003E'] = df5['B25044_003E'].apply(changeInteger)
df5['B25044_010E'] = df5['B25044_010E'].apply(changeInteger)
df5['B25044_001E'] = df5['B25044_001E'].apply(changeInteger)

In [33]:
df5['percent_no_vehicle'] = ((df5['B25044_003E'] + df5['B25044_010E'])/df5['B25044_001E'])*100

In [34]:
df5['fips'] = df5['state code'] + df5['county code']
df5.head()

,county_name,B25044_003E,B25044_010E,B25044_001E,state code,county code,percent_no_vehicle,fips
0,"Autauga County, Alabama",410,757,21559,01,001,5.413053,01001
1,"Baldwin County, Alabama",1283,1344,84047,01,003,3.125632,01003
2,"Barbour County, Alabama",289,750,9322,01,005,11.145677,01005
3,"Bibb County, Alabama",104,377,7259,01,007,6.626257,01007
4,"Blount County, Alabama",395,682,21205,01,009,5.078991,01009


#### 6.Percent of Population Unemployed:sum(B21005_006E, B21005_007E, B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E)/B21005_001E

In [35]:
variables = 'B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E'
df6 = collectAll_county(year,variables)

['NAME', 'B21005_006E', 'B21005_007E', 'B21005_011E', 'B21005_012E', 'B21005_017E', 'B21005_022E', 'B21005_028E', 'B21005_029E', 'B21005_033E', 'B21005_034E', 'B21005_001E', 'state', 'county']


In [36]:
df6.shape

(3221, 14)

In [37]:
df6['B21005_006E'] = df6['B21005_006E'].apply(changeInteger)
df6['B21005_007E'] = df6['B21005_007E'].apply(changeInteger)
df6['B21005_011E'] = df6['B21005_011E'].apply(changeInteger)
df6['B21005_012E'] = df6['B21005_012E'].apply(changeInteger)
df6['B21005_017E'] = df6['B21005_017E'].apply(changeInteger)
df6['B21005_022E'] = df6['B21005_022E'].apply(changeInteger)
df6['B21005_028E'] = df6['B21005_028E'].apply(changeInteger)
df6['B21005_029E'] = df6['B21005_029E'].apply(changeInteger)
df6['B21005_033E'] = df6['B21005_033E'].apply(changeInteger)
df6['B21005_034E'] = df6['B21005_034E'].apply(changeInteger)
df6['B21005_001E'] = df6['B21005_001E'].apply(changeInteger)

In [38]:
df6['percent_unemployed_CDC'] = ((df6['B21005_006E']+df6['B21005_007E']+df6['B21005_011E']+df6['B21005_012E']+df6['B21005_017E']+df6['B21005_022E']+df6['B21005_028E']+df6['B21005_029E']+df6['B21005_033E']+df6['B21005_034E'])/df6['B21005_001E'])*100

In [39]:
df6['fips'] = df6['state code'] + df6['county code']
df6.head()

,county_name,B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E,state code,county code,percent_unemployed_CDC,fips
0,"Autauga County, Alabama",38,30,378,2566,4,226,16,133,47,2856,33297,01,001,18.902604,01001
1,"Baldwin County, Alabama",196,324,1567,8970,136,1176,17,1131,548,11915,126259,01,003,20.576751,01003
2,"Barbour County, Alabama",0,18,353,2308,11,221,0,110,29,1568,15027,01,005,30.731350,01005
3,"Bibb County, Alabama",0,19,422,1933,0,184,0,142,6,1378,14114,01,007,28.935808,01007
4,"Blount County, Alabama",3,17,444,3652,162,392,0,453,200,3096,33957,01,009,24.793121,01009


#### 7.Percent of Population Living In Accommodations with Less Than 1 Room Per Person/Crowding: (B25014_005E+B25014_006E+B25014_007E+B25014_011E+B25014_012E+B25014_013E)/B25014_001E

In [40]:
variables = 'B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E'
df7 = collectAll_county(year,variables)

['NAME', 'B25014_005E', 'B25014_006E', 'B25014_007E', 'B25014_011E', 'B25014_012E', 'B25014_013E', 'B25014_001E', 'state', 'county']


In [41]:
df7.shape

(3221, 10)

In [42]:
df7['B25014_005E'] = df7['B25014_005E'].apply(changeInteger)
df7['B25014_006E'] = df7['B25014_006E'].apply(changeInteger)
df7['B25014_007E'] = df7['B25014_007E'].apply(changeInteger)
df7['B25014_011E'] = df7['B25014_011E'].apply(changeInteger)
df7['B25014_012E'] = df7['B25014_012E'].apply(changeInteger)
df7['B25014_013E'] = df7['B25014_013E'].apply(changeInteger)
df7['B25014_001E'] = df7['B25014_001E'].apply(changeInteger)

In [43]:
df7['percent_overcrowding'] = ((df7['B25014_005E']+df7['B25014_006E']+df7['B25014_007E']+df7['B25014_011E']+df7['B25014_012E']+df7['B25014_013E'])/df7['B25014_001E'])*100

In [44]:
df7['fips'] = df7['state code'] + df7['county code']
df7.head()

,county_name,B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E,state code,county code,percent_overcrowding,fips
0,"Autauga County, Alabama",190,76,37,28,0,8,21559,01,001,1.572429,01001
1,"Baldwin County, Alabama",447,393,51,226,83,80,84047,01,003,1.522957,01003
2,"Barbour County, Alabama",191,13,10,109,23,4,9322,01,005,3.754559,01005
3,"Bibb County, Alabama",0,54,10,27,0,0,7259,01,007,1.253616,01007
4,"Blount County, Alabama",232,27,16,105,5,0,21205,01,009,1.815610,01009


#### 8.Percent of Housing Units with 10+ Units in Structure: (B25024_007E+B25024_008E+B25024_009E)/B25024_001E

In [45]:
variables = 'B25024_007E,B25024_008E,B25024_009E,B25024_001E'
df8 = collectAll_county(year,variables)

['NAME', 'B25024_007E', 'B25024_008E', 'B25024_009E', 'B25024_001E', 'state', 'county']


In [46]:
df8.shape

(3221, 7)

In [47]:
df8['B25024_007E'] = df8['B25024_007E'].apply(changeInteger)
df8['B25024_008E'] = df8['B25024_008E'].apply(changeInteger)
df8['B25024_009E'] = df8['B25024_009E'].apply(changeInteger)
df8['B25024_001E'] = df8['B25024_001E'].apply(changeInteger)

In [48]:
df8['percent_multi_unit_housing'] = ((df8['B25024_007E']+df8['B25024_008E']+df8['B25024_009E'])/df8['B25024_001E'])*100

In [49]:
df8['fips'] = df8['state code'] + df8['county code']
df8.head()

,county_name,B25024_007E,B25024_008E,B25024_009E,B25024_001E,state code,county code,percent_multi_unit_housing,fips
0,"Autauga County, Alabama",505,291,122,23697,01,001,3.873908,01001
1,"Baldwin County, Alabama",3960,2838,12715,116747,01,003,16.713920,01003
2,"Barbour County, Alabama",80,33,57,12057,01,005,1.409969,01005
3,"Bibb County, Alabama",75,95,58,9237,01,007,2.468334,01007
4,"Blount County, Alabama",72,78,17,24404,01,009,0.684314,01009


#### 9.Percent Of Population 65+: (B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_049E)/B01001_001E

In [50]:
variables = 'B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E'
df9 = collectAll_county(year,variables)

['NAME', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E', 'B01001_001E', 'state', 'county']


In [51]:
df9.shape

(3221, 16)

In [52]:
df9['B01001_020E'] = df9['B01001_020E'].apply(changeInteger)
df9['B01001_021E'] = df9['B01001_021E'].apply(changeInteger)
df9['B01001_022E'] = df9['B01001_022E'].apply(changeInteger)
df9['B01001_023E'] = df9['B01001_023E'].apply(changeInteger)
df9['B01001_024E'] = df9['B01001_024E'].apply(changeInteger)
df9['B01001_025E'] = df9['B01001_025E'].apply(changeInteger)
df9['B01001_044E'] = df9['B01001_044E'].apply(changeInteger)
df9['B01001_045E'] = df9['B01001_045E'].apply(changeInteger)
df9['B01001_046E'] = df9['B01001_046E'].apply(changeInteger)
df9['B01001_047E'] = df9['B01001_047E'].apply(changeInteger)
df9['B01001_048E'] = df9['B01001_048E'].apply(changeInteger)
df9['B01001_049E'] = df9['B01001_049E'].apply(changeInteger)
df9['B01001_001E'] = df9['B01001_001E'].apply(changeInteger)

In [53]:
df9['percent_age_65_and_older'] = ((df9['B01001_020E']+df9['B01001_021E']+df9['B01001_022E']+df9['B01001_023E']+df9['B01001_024E']+df9['B01001_025E']+df9['B01001_044E']
                                  +df9['B01001_045E']+df9['B01001_046E']+df9['B01001_047E']+df9['B01001_048E']+df9['B01001_049E'])/df9['B01001_001E'])*100

In [54]:
df9['fips'] = df9['state code'] + df9['county code']
df9.head()

,county_name,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E,state code,county code,percent_age_65_and_older,fips
0,"Autauga County, Alabama",429,699,1077,764,389,325,496,836,1414,1066,358,637,55639,01,001,15.259081,01001
1,"Baldwin County, Alabama",2762,3855,6007,4273,2163,1541,3058,4236,7008,4669,2556,2588,218289,01,003,20.484770,01003
2,"Barbour County, Alabama",301,327,674,429,200,142,222,566,766,474,335,341,25026,01,005,19.088148,01005
3,"Bibb County, Alabama",212,457,388,352,104,104,71,461,580,451,272,224,22374,01,007,16.429785,01007
4,"Blount County, Alabama",476,1073,1297,630,752,407,708,1050,1415,868,856,850,57755,01,009,17.975933,01009


#### 10.Percent Of Population Who Do Not Speak English: B99163_005E/B99163_001E

In [55]:
variables = 'B99163_005E,B99163_001E'
df10 = collectAll_county(year,variables)

['NAME', 'B99163_005E', 'B99163_001E', 'state', 'county']


In [56]:
df10.shape

(3221, 5)

In [57]:
df10['B99163_005E'] = df10['B99163_005E'].apply(changeInteger)
df10['B99163_001E'] = df10['B99163_001E'].apply(changeInteger)

In [58]:
df10['percent_limited_english_abilities'] = (df10['B99163_005E']/df10['B99163_001E'])*100

In [59]:
df10['fips'] = df10['state code'] + df10['county code']
df10.head()

,county_name,B99163_005E,B99163_001E,state code,county code,percent_limited_english_abilities,fips
0,"Autauga County, Alabama",1877,52404,01,001,3.581788,01001
1,"Baldwin County, Alabama",9039,206329,01,003,4.380867,01003
2,"Barbour County, Alabama",1609,23694,01,005,6.790749,01005
3,"Bibb County, Alabama",450,21121,01,007,2.130581,01007
4,"Blount County, Alabama",3657,54250,01,009,6.741014,01009


#### 11.Percent of Population 17 Years of Age and Under: (B01001_003E+B01001_004E+B01001_005E+B01001_006E+B01001_027E+B01001_028E+B01001_029E+B01001_030E)/B01003_001E

In [60]:
variables = 'B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E'
df11 = collectAll_county(year,variables)

['NAME', 'B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E', 'B01003_001E', 'state', 'county']


In [61]:
df11.shape

(3221, 12)

In [62]:
df11['B01001_003E'] = df11['B01001_003E'].apply(changeInteger)
df11['B01001_004E'] = df11['B01001_004E'].apply(changeInteger)
df11['B01001_005E'] = df11['B01001_005E'].apply(changeInteger)
df11['B01001_006E'] = df11['B01001_006E'].apply(changeInteger)
df11['B01001_027E'] = df11['B01001_027E'].apply(changeInteger)
df11['B01001_028E'] = df11['B01001_028E'].apply(changeInteger)
df11['B01001_029E'] = df11['B01001_029E'].apply(changeInteger)
df11['B01001_030E'] = df11['B01001_030E'].apply(changeInteger)
df11['B01003_001E'] = df11['B01003_001E'].apply(changeInteger)

In [63]:
df11['percent_age_17_and_younger'] = ((df11['B01001_003E']+df11['B01001_004E']+df11['B01001_005E']+df11['B01001_006E']+df11['B01001_027E']+df11['B01001_028E']+df11['B01001_029E']+df11['B01001_030E'])/df11['B01003_001E'])*100

In [64]:
df11['fips'] = df11['state code'] + df11['county code']
df11.head()

,county_name,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E,state code,county code,percent_age_17_and_younger,fips
0,"Autauga County, Alabama",1727,2156,1674,1208,1508,1569,1971,1330,55639,01,001,23.621920,01001
1,"Baldwin County, Alabama",6082,5197,8289,4305,5878,6362,6882,3998,218289,01,003,21.527883,01003
2,"Barbour County, Alabama",660,741,721,467,672,809,708,444,25026,01,005,20.866299,01005
3,"Bibb County, Alabama",626,693,621,569,627,493,577,378,22374,01,007,20.488067,01007
4,"Blount County, Alabama",1844,1738,2034,1257,1661,1748,2043,1047,57755,01,009,23.152974,01009


#### 12.Percent Minority: 1-(B03002_003E/B03002_001E)

In [65]:
variables = 'B03002_003E,B03002_001E'
df12 = collectAll_county(year,variables)

['NAME', 'B03002_003E', 'B03002_001E', 'state', 'county']


In [66]:
df12.shape

(3221, 5)

In [67]:
df12['B03002_003E'] = df12['B03002_003E'].apply(changeInteger)
df12['B03002_001E'] = df12['B03002_001E'].apply(changeInteger)

In [68]:
df12['percent_minorities'] = (1 - (df12['B03002_003E']/df12['B03002_001E']))*100

In [69]:
df12['fips'] = df12['state code'] + df12['county code']
df12.head()

,county_name,B03002_003E,B03002_001E,state code,county code,percent_minorities,fips
0,"Autauga County, Alabama",41160,55639,01,001,26.023113,01001
1,"Baldwin County, Alabama",180955,218289,01,003,17.103015,01003
2,"Barbour County, Alabama",11332,25026,01,005,54.719092,01005
3,"Bibb County, Alabama",16650,22374,01,007,25.583266,01007
4,"Blount County, Alabama",50065,57755,01,009,13.314865,01009


#### 13.Percent of Population with a Disability: (B18101_004E+B18101_007E+B18101_010E+B18101_013E+B18101_016E+B18101_019E)/B18101_002E + (B18101_023E+B18101_026E+B18101_029E+B18101_032E+B18101_035E+B18101_038E)/B18101_021E

In [70]:
variables = 'B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E'
df13 = collectAll_county(year,variables)

['NAME', 'B18101_004E', 'B18101_007E', 'B18101_010E', 'B18101_013E', 'B18101_016E', 'B18101_019E', 'B18101_002E', 'B18101_023E', 'B18101_026E', 'B18101_029E', 'B18101_032E', 'B18101_035E', 'B18101_038E', 'B18101_021E', 'state', 'county']


In [71]:
df13.shape

(3221, 17)

In [72]:
df13['B18101_004E'] = df13['B18101_004E'].apply(changeInteger)
df13['B18101_007E'] = df13['B18101_007E'].apply(changeInteger)
df13['B18101_010E'] = df13['B18101_010E'].apply(changeInteger)
df13['B18101_013E'] = df13['B18101_013E'].apply(changeInteger)
df13['B18101_016E'] = df13['B18101_016E'].apply(changeInteger)
df13['B18101_002E'] = df13['B18101_002E'].apply(changeInteger)
df13['B18101_023E'] = df13['B18101_023E'].apply(changeInteger)
df13['B18101_026E'] = df13['B18101_026E'].apply(changeInteger)
df13['B18101_029E'] = df13['B18101_029E'].apply(changeInteger)
df13['B18101_032E'] = df13['B18101_032E'].apply(changeInteger)
df13['B18101_035E'] = df13['B18101_035E'].apply(changeInteger)
df13['B18101_038E'] = df13['B18101_038E'].apply(changeInteger)
df13['B18101_021E'] = df13['B18101_021E'].apply(changeInteger)

In [73]:
df13['percent_disabled'] = ((df13['B18101_004E']+df13['B18101_007E']+df13['B18101_010E']+df13['B18101_013E']+df13['B18101_016E'])/df13['B18101_002E'] + (df13['B18101_023E']+df13['B18101_026E']+df13['B18101_029E']+df13['B18101_032E']+df13['B18101_035E']+df13['B18101_038E'])/df13['B18101_021E'])*100

In [74]:
df13['fips'] = df13['state code'] + df13['county code']
df13.head()

,county_name,B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E,state code,county code,percent_disabled,fips
0,"Autauga County, Alabama",0,637,478,1689,788,948,26244,0,392,509,2064,898,1255,28309,01,001,31.765994,01001
1,"Baldwin County, Alabama",52,1348,1652,5109,3689,3514,103821,31,558,1520,5851,2921,4370,111848,01,003,25.049345,01003
2,"Barbour County, Alabama",10,183,164,677,403,422,10362,0,143,153,859,436,709,11754,01,005,33.435786,01005
3,"Bibb County, Alabama",5,109,547,1047,179,253,10507,0,62,222,656,235,433,10241,01,007,33.661047,01007
4,"Blount County, Alabama",0,289,252,1953,945,823,28418,0,155,364,1898,674,1211,28856,01,009,27.009996,01009


#### 14.Percent of Population Living in Group Quarters: B26001_001E/B01003_001E

In [75]:
variables = 'B26001_001E,B01003_001E'
df14 = collectAll_county(year,variables)

['NAME', 'B26001_001E', 'B01003_001E', 'state', 'county']


In [76]:
df14.shape

(3221, 5)

In [77]:
df14['B26001_001E'] = df14['B26001_001E'].apply(changeInteger)
df14['B01003_001E'] = df14['B01003_001E'].apply(changeInteger)

In [78]:
df14['percent_institutionalized_in_group_quarters'] = (df14['B26001_001E']/df14['B01003_001E'])*100

In [79]:
df14['fips'] = df14['state code'] + df14['county code']
df14.head()

,county_name,B26001_001E,B01003_001E,state code,county code,percent_institutionalized_in_group_quarters,fips
0,"Autauga County, Alabama",578,55639,01,001,1.038840,01001
1,"Baldwin County, Alabama",2954,218289,01,003,1.353252,01003
2,"Barbour County, Alabama",2910,25026,01,005,11.627907,01005
3,"Bibb County, Alabama",1657,22374,01,007,7.405918,01007
4,"Blount County, Alabama",564,57755,01,009,0.976539,01009


#### 15.Percent of Children Living in Single Parent Households: (B09005_004E + b98005_005E)/B09005_001E
##### B09005_001E: total
##### B09005_004E: 	Estimate!!Total:!!In male householder, no spouse/partner present household
##### B09005_005E: Estimate!!Total:!!In female householder, no spouse/partner present household

In [80]:
variables = 'B09005_004E,B09005_005E,B09005_001E'
df15 = collectAll_county(year,variables)

['NAME', 'B09005_004E', 'B09005_005E', 'B09005_001E', 'state', 'county']


In [81]:
df15['B09005_001E'] = df15['B09005_001E'].apply(changeInteger)
df15['B09005_004E'] = df15['B09005_004E'].apply(changeInteger)
df15['B09005_005E'] = df15['B09005_005E'].apply(changeInteger)

In [82]:
df15['percent_single_parent_households_CDC'] = ((df15['B09005_004E'] + df15['B09005_005E'])/df15['B09005_001E'])*100

In [83]:
df15['fips'] = df15['state code'] + df15['county code']
df15.head()

,county_name,B09005_004E,B09005_005E,B09005_001E,state code,county code,percent_single_parent_households_CDC,fips
0,"Autauga County, Alabama",642,2986,13143,01,001,27.604048,01001
1,"Baldwin County, Alabama",1603,7010,46902,01,003,18.363822,01003
2,"Barbour County, Alabama",225,2597,5222,01,005,54.040597,01005
3,"Bibb County, Alabama",507,1159,4539,01,007,36.704120,01007
4,"Blount County, Alabama",876,2136,13352,01,009,22.558418,01009


### age 18 to 34
#### (B01001_007E+B01001_008E+B01001_009E+B01001_010E+B01001_011E+B01001_012E+B01001_031E+B01001_032E+B01001_033E+B01001_034E+B01001_035E+B01001_036E)/B01001_001E

In [84]:
variables = 'B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E'
df16 = collectAll_county(year,variables)

['NAME', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E', 'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E', 'B01001_035E', 'B01001_036E', 'B01001_001E', 'state', 'county']


In [85]:
df16['B01001_007E'] = df16['B01001_007E'].apply(changeInteger)
df16['B01001_008E'] = df16['B01001_008E'].apply(changeInteger)
df16['B01001_009E'] = df16['B01001_009E'].apply(changeInteger)
df16['B01001_010E'] = df16['B01001_010E'].apply(changeInteger)
df16['B01001_011E'] = df16['B01001_011E'].apply(changeInteger)
df16['B01001_012E'] = df16['B01001_012E'].apply(changeInteger)
df16['B01001_031E'] = df16['B01001_031E'].apply(changeInteger)
df16['B01001_032E'] = df16['B01001_032E'].apply(changeInteger)
df16['B01001_033E'] = df16['B01001_033E'].apply(changeInteger)
df16['B01001_034E'] = df16['B01001_034E'].apply(changeInteger)
df16['B01001_035E'] = df16['B01001_035E'].apply(changeInteger)
df16['B01001_036E'] = df16['B01001_036E'].apply(changeInteger)
df16['B01001_001E'] = df16['B01001_001E'].apply(changeInteger)

In [86]:
df16['percent_age_18_to_34'] = ((df16['B01001_007E'] + df16['B01001_008E'] +df16['B01001_009E'] +df16['B01001_010E'] +df16['B01001_011E']+df16['B01001_012E']+df16['B01001_031E']+df16['B01001_032E']+df16['B01001_033E']+df16['B01001_034E']+df16['B01001_035E']+df16['B01001_036E'])/df16['B01001_001E'])*100

In [87]:
df16['fips'] = df16['state code'] + df16['county code']
df16.head()

,county_name,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E,state code,county code,percent_age_18_to_34,fips
0,"Autauga County, Alabama",553,395,225,1127,1789,1669,625,224,306,1181,1870,1750,55639,01,001,21.053578,01001
1,"Baldwin County, Alabama",2406,1177,792,3581,6064,5977,2296,1245,1123,3113,6220,6126,218289,01,003,18.379304,01003
2,"Barbour County, Alabama",261,365,103,432,1150,1084,214,138,54,440,696,713,25026,01,005,22.576520,01005
3,"Bibb County, Alabama",323,111,71,524,1023,1011,193,134,126,321,744,470,22374,01,007,22.575311,01007
4,"Blount County, Alabama",743,378,312,1023,1801,1612,628,325,203,957,1782,1684,57755,01,009,19.821660,01009


### Age 35 to 44
#### (B01001_013E+B01001_014E+B01001_037E+B01001_038E)/B01001_001E

In [88]:
variables = 'B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E'
df17 = collectAll_county(year,variables)

['NAME', 'B01001_013E', 'B01001_014E', 'B01001_037E', 'B01001_038E', 'B01001_001E', 'state', 'county']


In [89]:
df17['B01001_013E'] = df17['B01001_013E'].apply(changeInteger)
df17['B01001_014E'] = df17['B01001_014E'].apply(changeInteger)
df17['B01001_037E'] = df17['B01001_037E'].apply(changeInteger)
df17['B01001_038E'] = df17['B01001_038E'].apply(changeInteger)
df17['B01001_001E'] = df17['B01001_001E'].apply(changeInteger)

In [90]:
df17['percent_age_35_to_44'] = ((df17['B01001_013E']+df17['B01001_014E'] +df17['B01001_037E']+df17['B01001_038E'])/df17['B01001_001E'])*100

In [91]:
df17['fips'] = df17['state code'] + df17['county code']
df17.head()

,county_name,B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E,state code,county code,percent_age_35_to_44,fips
0,"Autauga County, Alabama",2043,1394,1850,2017,55639,01,001,13.127483,01001
1,"Baldwin County, Alabama",6787,5979,7022,6623,218289,01,003,12.099098,01003
2,"Barbour County, Alabama",928,856,694,569,25026,01,005,12.175338,01005
3,"Bibb County, Alabama",643,989,922,531,22374,01,007,13.788326,01007
4,"Blount County, Alabama",1625,1896,1519,2020,57755,01,009,12.224050,01009


### age 45 to 54
#### (B01001_015E+B01001_016E+B01001_039E+B01001_040E)/B01001_001E

In [92]:
variables = 'B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E'
df18 = collectAll_county(year,variables)

['NAME', 'B01001_015E', 'B01001_016E', 'B01001_039E', 'B01001_040E', 'B01001_001E', 'state', 'county']


In [93]:
df18['B01001_015E'] = df18['B01001_015E'].apply(changeInteger)
df18['B01001_016E'] = df18['B01001_016E'].apply(changeInteger)
df18['B01001_039E'] = df18['B01001_039E'].apply(changeInteger)
df18['B01001_040E'] = df18['B01001_040E'].apply(changeInteger)
df18['B01001_001E'] = df18['B01001_001E'].apply(changeInteger)

In [94]:
df18['percent_age_45_to_54'] = ((df18['B01001_015E']+df18['B01001_016E'] +df18['B01001_039E']+df18['B01001_040E'])/df18['B01001_001E'])*100

In [95]:
df18['fips'] = df18['state code'] + df18['county code']
df18.head()

,county_name,B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E,state code,county code,percent_age_45_to_54,fips
0,"Autauga County, Alabama",2015,1937,1990,1874,55639,01,001,14.047700,01001
1,"Baldwin County, Alabama",6763,7037,7374,7396,218289,01,003,13.088154,01003
2,"Barbour County, Alabama",872,839,659,755,25026,01,005,12.487014,01005
3,"Bibb County, Alabama",841,896,771,627,22374,01,007,14.011799,01007
4,"Blount County, Alabama",1936,1858,1992,1844,57755,01,009,13.210977,01009


### age 55 to 64
#### (B01001_017E+B01001_018E+B01001_019E+B01001_041E+B01001_042E+B01001_043E)/B01001_001E

In [96]:
variables = 'B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E'
df19 = collectAll_county(year,variables)

['NAME', 'B01001_017E', 'B01001_018E', 'B01001_019E', 'B01001_041E', 'B01001_042E', 'B01001_043E', 'B01001_001E', 'state', 'county']


In [97]:
df19['B01001_017E'] = df19['B01001_017E'].apply(changeInteger)
df19['B01001_018E'] = df19['B01001_018E'].apply(changeInteger)
df19['B01001_019E'] = df19['B01001_019E'].apply(changeInteger)
df19['B01001_041E'] = df19['B01001_041E'].apply(changeInteger)
df19['B01001_042E'] = df19['B01001_042E'].apply(changeInteger)
df19['B01001_043E'] = df19['B01001_043E'].apply(changeInteger)
df19['B01001_001E'] = df19['B01001_001E'].apply(changeInteger)

In [98]:
df19['percent_age_55_to_64'] = ((df19['B01001_017E']+df19['B01001_018E'] +df19['B01001_019E']+df19['B01001_041E']+df19['B01001_042E']+df19['B01001_043E'])/df19['B01001_001E'])*100

In [99]:
df19['fips'] = df19['state code'] + df19['county code']
df19.head()

,county_name,B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E,state code,county code,percent_age_55_to_64,fips
0,"Autauga County, Alabama",2025,628,804,2080,835,800,55639,01,001,12.890239,01001
1,"Baldwin County, Alabama",7434,2273,5145,7802,3702,5123,218289,01,003,14.420791,01003
2,"Barbour County, Alabama",809,215,580,782,360,459,25026,01,005,12.806681,01005
3,"Bibb County, Alabama",857,304,303,649,286,444,22374,01,007,12.706713,01007
4,"Blount County, Alabama",2168,715,1102,1767,893,1218,57755,01,009,13.614406,01009


### Combine 19 dataframes together

In [100]:
df = df1[['fips','per_capita_income']].merge(df2[['fips', 'percent_below_poverty']], on = 'fips', how = 'left')

In [101]:
df = df.merge(df3[['fips', 'percent_no_highschool_diploma']], on = 'fips', how = 'left')
df = df.merge(df4[['fips', 'percent_mobile_homes']], on = 'fips', how = 'left')
df = df.merge(df5[['fips', 'percent_no_vehicle']], on = 'fips', how = 'left')
df = df.merge(df6[['fips', 'percent_unemployed_CDC']], on = 'fips', how = 'left')
df = df.merge(df7[['fips', 'percent_overcrowding']], on = 'fips', how = 'left')
df = df.merge(df8[['fips', 'percent_multi_unit_housing']], on = 'fips', how = 'left')
df = df.merge(df9[['fips', 'percent_age_65_and_older']], on = 'fips', how = 'left')
df = df.merge(df10[['fips', 'percent_limited_english_abilities']], on = 'fips', how = 'left')
df = df.merge(df11[['fips', 'percent_age_17_and_younger']], on = 'fips', how = 'left')
df = df.merge(df12[['fips', 'percent_minorities']], on = 'fips', how = 'left')
df = df.merge(df13[['fips', 'percent_disabled']], on = 'fips', how = 'left')
df = df.merge(df14[['fips', 'percent_institutionalized_in_group_quarters']], on = 'fips', how = 'left')
df = df.merge(df15[['fips', 'percent_single_parent_households_CDC']], on = 'fips', how = 'left')
df = df.merge(df16[['fips', 'percent_age_18_to_34']], on = 'fips', how = 'left')
df = df.merge(df17[['fips', 'percent_age_35_to_44']], on = 'fips', how = 'left')
df = df.merge(df18[['fips', 'percent_age_45_to_54']], on = 'fips', how = 'left')
df = df.merge(df19[['fips', 'percent_age_55_to_64']], on = 'fips', how = 'left')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,01001,29804,15.211758,11.640254,16.668786,5.413053,18.902604,1.572429,3.873908,15.259081,3.581788,23.621920,26.023113,31.765994,1.038840,27.604048,21.053578,13.127483,14.047700,12.890239
1,01003,33751,9.173677,10.039020,10.892331,3.125632,20.576751,1.522957,16.713920,20.484770,4.380867,21.527883,17.103015,25.049345,1.353252,18.363822,18.379304,12.099098,13.088154,14.420791
2,01005,20074,28.599909,26.150475,27.663230,11.145677,30.731350,3.754559,1.409969,19.088148,6.790749,20.866299,54.719092,33.435786,11.627907,54.040597,22.576520,12.175338,12.487014,12.806681
3,01007,22626,18.098130,20.347783,31.944780,6.626257,28.935808,1.253616,2.468334,16.429785,2.130581,20.488067,25.583266,33.661047,7.405918,36.704120,22.575311,13.788326,14.011799,12.706713
4,01009,25457,13.736062,18.448285,22.465073,5.078991,24.793121,1.815610,0.684314,17.975933,6.741014,23.152974,13.314865,27.009996,0.976539,22.558418,19.821660,12.224050,13.210977,13.614406


In [102]:
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,01001,29804,15.211758,11.640254,16.668786,5.413053,18.902604,1.572429,3.873908,15.259081,3.581788,23.621920,26.023113,31.765994,1.038840,27.604048,21.053578,13.127483,14.047700,12.890239
1,01003,33751,9.173677,10.039020,10.892331,3.125632,20.576751,1.522957,16.713920,20.484770,4.380867,21.527883,17.103015,25.049345,1.353252,18.363822,18.379304,12.099098,13.088154,14.420791
2,01005,20074,28.599909,26.150475,27.663230,11.145677,30.731350,3.754559,1.409969,19.088148,6.790749,20.866299,54.719092,33.435786,11.627907,54.040597,22.576520,12.175338,12.487014,12.806681
3,01007,22626,18.098130,20.347783,31.944780,6.626257,28.935808,1.253616,2.468334,16.429785,2.130581,20.488067,25.583266,33.661047,7.405918,36.704120,22.575311,13.788326,14.011799,12.706713
4,01009,25457,13.736062,18.448285,22.465073,5.078991,24.793121,1.815610,0.684314,17.975933,6.741014,23.152974,13.314865,27.009996,0.976539,22.558418,19.821660,12.224050,13.210977,13.614406


In [103]:
df = df.sort_values('fips')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,01001,29804,15.211758,11.640254,16.668786,5.413053,18.902604,1.572429,3.873908,15.259081,3.581788,23.621920,26.023113,31.765994,1.038840,27.604048,21.053578,13.127483,14.047700,12.890239
1,01003,33751,9.173677,10.039020,10.892331,3.125632,20.576751,1.522957,16.713920,20.484770,4.380867,21.527883,17.103015,25.049345,1.353252,18.363822,18.379304,12.099098,13.088154,14.420791
2,01005,20074,28.599909,26.150475,27.663230,11.145677,30.731350,3.754559,1.409969,19.088148,6.790749,20.866299,54.719092,33.435786,11.627907,54.040597,22.576520,12.175338,12.487014,12.806681
3,01007,22626,18.098130,20.347783,31.944780,6.626257,28.935808,1.253616,2.468334,16.429785,2.130581,20.488067,25.583266,33.661047,7.405918,36.704120,22.575311,13.788326,14.011799,12.706713
4,01009,25457,13.736062,18.448285,22.465073,5.078991,24.793121,1.815610,0.684314,17.975933,6.741014,23.152974,13.314865,27.009996,0.976539,22.558418,19.821660,12.224050,13.210977,13.614406


In [104]:
df.columns

Index(['fips', 'per_capita_income', 'percent_below_poverty',
       'percent_no_highschool_diploma', 'percent_mobile_homes',
       'percent_no_vehicle', 'percent_unemployed_CDC', 'percent_overcrowding',
       'percent_multi_unit_housing', 'percent_age_65_and_older',
       'percent_limited_english_abilities', 'percent_age_17_and_younger',
       'percent_minorities', 'percent_disabled',
       'percent_institutionalized_in_group_quarters',
       'percent_single_parent_households_CDC', 'percent_age_18_to_34',
       'percent_age_35_to_44', 'percent_age_45_to_54', 'percent_age_55_to_64'],
      dtype='object')

In [105]:
df = df.drop('percent_age_17_and_younger', axis = 1)

In [106]:
df.columns

Index(['fips', 'per_capita_income', 'percent_below_poverty',
       'percent_no_highschool_diploma', 'percent_mobile_homes',
       'percent_no_vehicle', 'percent_unemployed_CDC', 'percent_overcrowding',
       'percent_multi_unit_housing', 'percent_age_65_and_older',
       'percent_limited_english_abilities', 'percent_minorities',
       'percent_disabled', 'percent_institutionalized_in_group_quarters',
       'percent_single_parent_households_CDC', 'percent_age_18_to_34',
       'percent_age_35_to_44', 'percent_age_45_to_54', 'percent_age_55_to_64'],
      dtype='object')

In [107]:
df.to_csv('Social_Economic_0918/Social_Economic_2020.csv')